In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import preprocessing
from sklearn.decomposition import PCA
#import data
df=pd.read_csv('dataProject.csv', header=0); 

X1=df.ix[:,2:154] #instances
#X1.head()
X=X1.values

# standardscale data
from sklearn import preprocessing
X_scaled=preprocessing.scale(X)

pca = PCA(n_components=2).fit(X_scaled)
X_2d = pca.transform(X_scaled)

#define lables
from sklearn.preprocessing import LabelEncoder
y = df.loc[:, 'Label'].values
#y

In [16]:
#set training set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
        train_test_split(X_2d, y, test_size=0.20, random_state=1)


In [17]:
print(pca.explained_variance_ratio_)  

[ 0.65467555  0.23347428]


In [18]:
print(pca.explained_variance_)

[ 99.51068333  35.48809009]


In [19]:
pca.noise_variance_

0.32694666509279369

In [20]:
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt
%matplotlib 

Using matplotlib backend: Qt5Agg


In [31]:
#=====================================================================================================
#Fig 3
#=====================================================================================================
pl.close('all')
pl.figure(figsize=(8,6))
h = .02  # step size in the mesh
#C = 10  # SVM regularization parameter
svc = svm.SVC(kernel='linear').fit(X_train, y_train)
rbf_svc = svm.SVC(kernel='rbf').fit(X_train, y_train)
poly_svc = svm.SVC(kernel='poly').fit(X_train, y_train)
sigmod_svc = svm.SVC(kernel='sigmoid').fit(X_train, y_train)

# create a mesh to plot in
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# title for the plots
titles = ['SVC with linear kernel',
          'SVC with sigmoid kernel',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 3) kernel']


for k, clf in enumerate((svc, sigmod_svc, rbf_svc, poly_svc)):
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    pl.subplot(2, 2, k + 1)
    pl.subplots_adjust(wspace=0.08, hspace=0.20)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    pl.contourf(xx, yy, Z, cmap=pl.cm.coolwarm, alpha=0.5)

    # Plot also the training points
    pl.scatter(X_2d[:, 0], X_2d[:, 1], c=y, cmap=pl.cm.coolwarm,s=20) 

    #Plot score
    
    pl.text(0.6,0.9, ('Test accuracy: %.2f' % clf.score(X_2d, y)), transform=pl.gca().transAxes,
            bbox={'facecolor':'white'})


    pl.xlabel('Principal component 1')
    pl.ylabel('Principal component 2')
    pl.xlim(xx.min(), xx.max())
    pl.ylim(yy.min(), yy.max())
    pl.xticks(())
    pl.yticks(())
    pl.title(titles[k])

pl.subplots_adjust(left=0.03, right=0.99, bottom=0.04, top=0.95)    
pl.show()


In [28]:
X_train, X_test, y_train, y_test = \
        train_test_split(X_2d, y, test_size=0.20, random_state=20)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

scores = cross_val_score(estimator=svm.SVC(kernel='rbf', gamma=0.01, C=10, random_state=69), 
                         X=X_train, 
                         y=y_train, 
                         cv=KFold(n_splits=10,random_state=1),
                         n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [ 1.    1.    1.    1.    1.    1.    1.    0.75  1.    1.  ]
CV accuracy: 0.975 +/- 0.075


In [29]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
%matplotlib 
X_train, X_test, y_train, y_test = \
        train_test_split(X_2d, y, test_size=0.20, random_state=20)

C_range = np.logspace(-5, 7, 13)
gamma_range = np.logspace(-8, 4, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = KFold(n_splits=10,shuffle=True,random_state=3)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.3f"
      % (grid.best_params_, grid.best_score_))

# Now we need to fit a classifier for all parameters in the 2d version
# (we use a smaller set of parameters here because it takes a while to train)



Using matplotlib backend: Qt5Agg
The best parameters are {'C': 10.0, 'gamma': 0.01} with a score of 1.000


In [30]:
#Fig 5
# Draw heatmap of the validation accuracy as a function of gamma and C
#
# The score are encoded as colors with the hot colormap which varies from dark
# red to bright yellow. As the most interesting scores are all located in the
# 0.92 to 0.97 range we use a custom normalizer to set the mid-point to 0.92 so
# as to make it easier to visualize the small variations of score values in the
# interesting range while not brutally collapsing all the low score values to
# the same color.

from matplotlib.colors import Normalize
class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))

X_train, X_test, y_train, y_test = \
        train_test_split(X_2d, y, test_size=0.20, random_state=20)
    
C_range = np.logspace(-5, 7, 13)
gamma_range = np.logspace(-8, 4, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = KFold(n_splits=10,shuffle=True,random_state=3)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X_train, y_train)   

scores = grid.cv_results_['mean_test_score'].reshape(len(C_range),
                                                     len(gamma_range))

plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot,
           norm=MidpointNormalize(vmin=0.1,midpoint=0.92))
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.title('K-Fold Cross Validation Score')
plt.show()


In [33]:
#==========================Figure 4
plt.close('all')
plt.figure(figsize=(8,6))
C_2d_range = [0.1,1,10,100,1000]
gamma_2d_range = [1e-4,1e-3, 1e-2,1e-1,1]
classifiers = []
for C in C_2d_range:
    for gamma in gamma_2d_range:
        clf = SVC(kernel='rbf', C=C, gamma=gamma)
        clf.fit(X_train, y_train)
        
        classifiers.append((C, gamma, clf))
h=.1
x_min, x_max = X_train[:, 0].min() - 2, X_train[:, 0].max() + 2
y_min, y_max = X_train[:, 1].min() - 2, X_train[:, 1].max() + 2
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
for (k, (C, gamma, clf)) in enumerate(classifiers):
    # evaluate decision function in a grid
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # visualize decision function for these parameters
    plt.subplot(len(C_2d_range), len(gamma_2d_range), k + 1)
    plt.subplots_adjust(wspace=0.07, hspace=0.27)
    plt.title(r'$\gamma$'"=%g, C=%g" % (gamma,C),
              size='medium',weight='bold')

    # visualize parameter's effect on decision function
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=pl.cm.coolwarm, alpha=0.5)
    #plt.pcolormesh(xx, yy, -Z, cmap=plt.cm.RdBu)
    #plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=pl.cm.coolwarm)
    #plt.scatter(X_test[:, 0], X_test[:, 1], y_test, cmap=pl.cm.RdBu_r)
    
    for i in range(0, X_train.shape[0]):
        if y_train[i] == 0:
            c1 = pl.scatter(X_train[i,0],X_train[i,1],c='r',marker='o', s = 20 )
        elif y_train[i] == 1:
            c2 = pl.scatter(X_train[i,0],X_train[i,1],c='g',marker='+', s = 30 )
    for i in range(0, X_test.shape[0]):
        if y_test[i] == 0:
            c3 = pl.scatter(X_test[i,0],X_test[i,1],c='b',marker='o', s = 20 )
        elif y_test[i] == 1:
            c4 = pl.scatter(X_test[i,0],X_test[i,1],c='m',marker='+', s = 30 )        
    plt.text(0.7,0.8, ('%.2f'% np.mean(cross_val_score(estimator=clf, 
                         X=X_train, 
                         y=y_train, 
                         cv=KFold(n_splits=10,shuffle=True,random_state=3),
                         n_jobs=1)) ), transform=pl.gca().transAxes,
            bbox={'facecolor':'white'})    
    
    plt.xticks(())
    plt.yticks(())
    

plt.subplots_adjust(left=0.02, right=0.98, bottom=0.01, top=0.96)


scores = grid.cv_results_['mean_test_score'].reshape(len(C_range),
                                                     len(gamma_range))


In [35]:
#Fig 5
# Draw heatmap of the validation accuracy as a function of gamma and C
#
# The score are encoded as colors with the hot colormap which varies from dark
# red to bright yellow. As the most interesting scores are all located in the
# 0.92 to 0.97 range we use a custom normalizer to set the mid-point to 0.92 so
# as to make it easier to visualize the small variations of score values in the
# interesting range while not brutally collapsing all the low score values to
# the same color.

scores = grid.cv_results_['mean_test_score'].reshape(len(C_range),
                                                     len(gamma_range))

plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=0, right=1, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot,
           norm=MidpointNormalize(vmin=0.2, midpoint=0.92))
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.title('K-Fold Cross Validation Score')
plt.show()

In [ ]:
#Overfit
gamma=1
C=10
clf = SVC(C=C, gamma=gamma)
clf.fit(X_train, y_train)
h=.1
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.title("gamma=10^%d, C=10^%d" % (np.log10(gamma), np.log10(C)),
              size='medium')

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=pl.cm.coolwarm, alpha=0.5)
for i in range(0, X_train.shape[0]):
    if y_train[i] == 0:
        c1 = pl.scatter(X_train[i,0],X_train[i,1],c='r',marker='o', s = 20 )
    elif y_train[i] == 1:
        c2 = pl.scatter(X_train[i,0],X_train[i,1],c='g',marker='+', s = 30 )
for i in range(0, X_test.shape[0]):
    if y_test[i] == 0:
        c3 = pl.scatter(X_test[i,0],X_test[i,1],c='b',marker='o', s = 20 )
    elif y_test[i] == 1:
        c4 = pl.scatter(X_test[i,0],X_test[i,1],c='m',marker='+', s = 30 )     
plt.xticks(())
plt.yticks(())
plt.axis('tight')
plt.xlabel('Overfitting')
pl.legend([c1, c2,c3,c4], ['Train Nnegative','Train Positive','test Nnegative','test Positive'])
    

In [ ]:
#good fit
gamma=1e-2
C=10
clf = SVC(C=C, gamma=gamma)
clf.fit(X_train, y_train)
h=.1
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.title("gamma=10^%d, C=10^%d" % (np.log10(gamma), np.log10(C)),
              size='medium')

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=pl.cm.coolwarm, alpha=0.5)
for i in range(0, X_train.shape[0]):
    if y_train[i] == 0:
        c1 = pl.scatter(X_train[i,0],X_train[i,1],c='r',marker='o', s = 20 )
    elif y_train[i] == 1:
        c2 = pl.scatter(X_train[i,0],X_train[i,1],c='g',marker='+', s = 30 )
for i in range(0, X_test.shape[0]):
    if y_test[i] == 0:
        c3 = pl.scatter(X_test[i,0],X_test[i,1],c='b',marker='o', s = 20 )
    elif y_test[i] == 1:
        c4 = pl.scatter(X_test[i,0],X_test[i,1],c='m',marker='+', s = 30 )     
plt.xticks(())
plt.yticks(())
plt.axis('tight')
plt.xlabel('Good Fitting')
pl.legend([c1, c2,c3,c4], ['Train Negative','Train Positive','Test Negative','Test Positive'])
    

In [ ]:
#underfit
gamma=1e-4
C=10
clf = SVC(C=C, gamma=gamma)
clf.fit(X_train, y_train)
h=.1
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.title("gamma=10^%d, C=10^%d" % (np.log10(gamma), np.log10(C)),
              size='medium')

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap=pl.cm.coolwarm, alpha=0.5)
for i in range(0, X_train.shape[0]):
    if y_train[i] == 0:
        c1 = pl.scatter(X_train[i,0],X_train[i,1],c='r',marker='o', s = 20 )
    elif y_train[i] == 1:
        c2 = pl.scatter(X_train[i,0],X_train[i,1],c='g',marker='+', s = 30 )
for i in range(0, X_test.shape[0]):
    if y_test[i] == 0:
        c3 = pl.scatter(X_test[i,0],X_test[i,1],c='b',marker='o', s = 20 )
    elif y_test[i] == 1:
        c4 = pl.scatter(X_test[i,0],X_test[i,1],c='m',marker='+', s = 30 )     
plt.xticks(())
plt.yticks(())
plt.axis('tight')
plt.xlabel('Under Fitting')
pl.legend([c1, c2,c3,c4], ['Train Nnegative','Train Positive','test Nnegative','test Positive'])
    